In [100]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col, col2im
import numpy as np

%matplotlib inline

# 7.1 畳み込みニューラルネットワーク
- 主に画像認識で用いられる
- Affineレイヤは全結合層とも呼ばれ，データの形状を無視．画像データ（3D）も1Dのデータにする．
- Convolutionレイヤ（畳み込み層）と Poolingレイヤ（プーリング層）が入る
- 畳み込み層では形状を維持できる

# 7.2 畳み込み層

### 畳み込み演算
- 入力データ（入力特徴マップ）に対して，フィルターを適用する（積和演算）
- バイアスは出力データ全てに対して同じ値が加算される
- 4x4の入力データに対し，3x3のフィルターをストライド1で適用する場合，出力データは2x2になる

### パディング
- 入力データの周囲に固定のデータを埋める
- 主に，出力データのサイズを調整するために使われる
- 4x4の入力データに対し，幅1のパディングを適用し，3x3のフィルターをストライド1で適用する場合，出力データは4x4になる

### ストライド
- フィルターを動かす幅

### 出力データのサイズの計算
- 入力サイズ: (H, W)
- フィルターサイズ: (FH, FW)
- 出力サイズ: (OH, OW)
- パディング: P
- ストライド: S
$$
OH = \frac{H + 2P - FH}{S} +1 \\\\
OW = \frac{W + 2P - FW}{S} +1
$$

### 3次元データの畳み込み演算
- 縦，横，チャンネル方向がある画像データなどの例を考える
- 入力データとフィルターのチャンネル数を揃える
- チャンネルごとの畳み込み演算結果を加算して一つの出力を得る
- データ，フィルターを(channel, height, width)の直方体で考えると，(C, H, W)の入力データと(C, FH, FW)のフィルターで(1, OH, OW)の出力データが得られる
- フィルターをFN個使うと，入力データ(C, H, W)，フィルター(FN, C, FH, FW), 出力データ(FN, OH, OW)になる．
- バイアスは出力データのチャンネルごとに一つの値が使われるため，(FN, 1, 1)の形状になる

### 畳み込み演算のバッチ処理
- 入力データの形状が(N, C, H, W)，出力データの形状が(N, FN, OH, OW)になる
- フィルター，バイアスはN個のデータに対して同じものが使われるため変わらない

# 7.3 プーリング層
- 縦・横方向の空間を小さくする演算
- 特定領域の最大値を取るMaxプーリング，平均を取るAverageプーリングなどがある
- 最大値や平均を取るだけなので，学習すべきパラメータはない
- チャンネルごとに計算するので，チャンネル数は変化しない
- 微小な変化にロバスト

# 7.4 Convolution / Pooling レイヤの実装

### im2col の挙動
- im2col(input_data, filter_h, filter_w, stride=1, pad=0)
- フィルターサイズ，ストライド，パディングを考慮して入力データを2次元に展開する

In [82]:
# インデックスを値に持つ配列を4次元に変換
x3 = np.arange(10*3*7*7).reshape(10, 3, 7, 7)
# im2colで展開
col3 = im2col(x3, 5, 5, stride=1, pad=0)

print('===フィルターに対応する部分===')
print(x3[0, :, :5, :5].flatten(), end='\n\n')

print('===im2col展開後の1行目===')
print(col3[0], end='\n\n')

print('===形状の変化===')
print('元データ: ', x3.shape)
print('im2colデータ: ', col3.shape, end='\n\n')

===フィルターに対応する部分===
[  0   1   2   3   4   7   8   9  10  11  14  15  16  17  18  21  22  23
  24  25  28  29  30  31  32  49  50  51  52  53  56  57  58  59  60  63
  64  65  66  67  70  71  72  73  74  77  78  79  80  81  98  99 100 101
 102 105 106 107 108 109 112 113 114 115 116 119 120 121 122 123 126 127
 128 129 130]

===im2col展開後の1行目===
[  0.   1.   2.   3.   4.   7.   8.   9.  10.  11.  14.  15.  16.  17.
  18.  21.  22.  23.  24.  25.  28.  29.  30.  31.  32.  49.  50.  51.
  52.  53.  56.  57.  58.  59.  60.  63.  64.  65.  66.  67.  70.  71.
  72.  73.  74.  77.  78.  79.  80.  81.  98.  99. 100. 101. 102. 105.
 106. 107. 108. 109. 112. 113. 114. 115. 116. 119. 120. 121. 122. 123.
 126. 127. 128. 129. 130.]

===形状の変化===
元データ:  (10, 3, 7, 7)
im2colデータ:  (90, 75)



In [83]:
# 1データ目，2データ目，，，の順に展開される
print('===1データ目の最後の出力に対応する部分===')
print(x3[0, :, -5:, -5:].flatten())
print('===展開後の9行目===')
print(col3[8], end='\n\n')
print('===2データ目の最初の出力に対応する部分===')
print(x3[1, :, :5, :5].flatten())
print('===展開後の10行目===')
print(col3[9])

===1データ目の最後の出力に対応する部分===
[ 16  17  18  19  20  23  24  25  26  27  30  31  32  33  34  37  38  39
  40  41  44  45  46  47  48  65  66  67  68  69  72  73  74  75  76  79
  80  81  82  83  86  87  88  89  90  93  94  95  96  97 114 115 116 117
 118 121 122 123 124 125 128 129 130 131 132 135 136 137 138 139 142 143
 144 145 146]
===展開後の9行目===
[ 16.  17.  18.  19.  20.  23.  24.  25.  26.  27.  30.  31.  32.  33.
  34.  37.  38.  39.  40.  41.  44.  45.  46.  47.  48.  65.  66.  67.
  68.  69.  72.  73.  74.  75.  76.  79.  80.  81.  82.  83.  86.  87.
  88.  89.  90.  93.  94.  95.  96.  97. 114. 115. 116. 117. 118. 121.
 122. 123. 124. 125. 128. 129. 130. 131. 132. 135. 136. 137. 138. 139.
 142. 143. 144. 145. 146.]

===2データ目の最初の出力に対応する部分===
[147 148 149 150 151 154 155 156 157 158 161 162 163 164 165 168 169 170
 171 172 175 176 177 178 179 196 197 198 199 200 203 204 205 206 207 210
 211 212 213 214 217 218 219 220 221 224 225 226 227 228 245 246 247 248
 249 252 253 254 255 256 259

In [85]:
# フィルター個数は1．今回はわかりやすく要素は全て1
W = np.ones((1, 3, 5, 5))
col_w = W.reshape(W.shape[0], -1).T
print('===形状の変化===')
print('元のフィルター:', W.shape)
print('展開後のフィルター:', col_w.shape)

===形状の変化===
元のフィルター: (1, 3, 5, 5)
展開後のフィルター: (75, 1)


In [77]:
# 出力の形状
out_h = int(1 + (7 + 2*0 - 5) / 1)
out_w = int(1 + (7 + 2*0 - 5) / 1)

out = np.dot(col3, col_w)
print('演算後:', out.shape)
print(out[:9], end='\n\n')

out = out.reshape(10, out_h, out_w, -1)
print('reshape後:', out.shape)
print(out[0], end='\n\n')

out = out.transpose(0, 3, 1, 2)
print('transpose後:', out.shape)
print(out[0], end='\n\n')

演算後: (90, 1)
[[4875.]
 [4950.]
 [5025.]
 [5400.]
 [5475.]
 [5550.]
 [5925.]
 [6000.]
 [6075.]]

reshape後: (10, 3, 3, 1)
[[[4875.]
  [4950.]
  [5025.]]

 [[5400.]
  [5475.]
  [5550.]]

 [[5925.]
  [6000.]
  [6075.]]]

transpose後: (10, 1, 3, 3)
[[[4875. 4950. 5025.]
  [5400. 5475. 5550.]
  [5925. 6000. 6075.]]]



### backward

In [105]:
# forwardの入力は(10, 3, 5, 5)としておく
x = np.arange(10*3*5*5).reshape(10, 3, 5, 5)
col = im2col(x, 2, 2, stride=1, pad=0)

print('xの形状: ', x.shape)
print('im2col後の形状: ', col.shape, end='\n\n')

# フィルターは(3, 3, 2, 2)としておく
W = np.ones((3, 3, 2, 2))
col_w = W.reshape(W.shape[0], -1).T

print('フィルターの形状: ', W.shape)
print('フィルター展開，転置後の形状: ', col_w.shape, end='\n\n')

# バイアスは(3, 1, 1)としておく
b = np.arange(3)
print('バイアスの形状: ', b.shape)

# 出力の形状
OH = int(1 + (5 + 2*0 - 2)/1)
OW = int(1 + (5 + 2*0 - 2)/1)

print('畳み込み演算後の形状: ', (np.dot(col, col_w) + b).reshape(10, OH, OW, -1).transpose(0, 3, 1, 2).shape, end='\n\n')

# forwardの出力が(10x3x4x4)とする=backwardのdoutも同じ
dout = np.arange(10*3*4*4).reshape(10, 3, 4, 4)
print('doutの形状: (N, FN, OH, OW) =', dout.shape)
print('一つ目のデータのdout')
print(dout[0])

xの形状:  (10, 3, 5, 5)
im2col後の形状:  (160, 12)

フィルターの形状:  (3, 3, 2, 2)
フィルター展開，転置後の形状:  (12, 3)

バイアスの形状:  (3,)
畳み込み演算後の形状:  (10, 3, 4, 4)

doutの形状: (N, FN, OH, OW) = (10, 3, 4, 4)
一つ目のデータのdout
[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]
  [12 13 14 15]]

 [[16 17 18 19]
  [20 21 22 23]
  [24 25 26 27]
  [28 29 30 31]]

 [[32 33 34 35]
  [36 37 38 39]
  [40 41 42 43]
  [44 45 46 47]]]


In [103]:
dout = dout.transpose(0, 2, 3, 1)
print('transpose後: (N, OH, OW, FN) =', dout.shape, end='\n\n')
print('一つ目のデータのtranspose後のdout')
print(dout[0], end='\n\n')

# ((NxOHxOW), FN)にreshape
dout = dout.reshape(-1, dout.shape[-1])
print('reshape後: ((N x OH x OW), FN) =', dout.shape, end='\n\n')
print('一つ目のデータのreshape後のdout')
print(dout[:, 0], end='\n\n')

transpose後: (N, OH, OW, FN) = (10, 4, 4, 3)

一つ目のデータのtranspose後のdout
[[[ 0 16 32]
  [ 1 17 33]
  [ 2 18 34]
  [ 3 19 35]]

 [[ 4 20 36]
  [ 5 21 37]
  [ 6 22 38]
  [ 7 23 39]]

 [[ 8 24 40]
  [ 9 25 41]
  [10 26 42]
  [11 27 43]]

 [[12 28 44]
  [13 29 45]
  [14 30 46]
  [15 31 47]]]

reshape後: ((N x OH x OW), FN) = (160, 3)

一つ目のデータのreshape後のdout
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  48  49
  50  51  52  53  54  55  56  57  58  59  60  61  62  63  96  97  98  99
 100 101 102 103 104 105 106 107 108 109 110 111 144 145 146 147 148 149
 150 151 152 153 154 155 156 157 158 159 192 193 194 195 196 197 198 199
 200 201 202 203 204 205 206 207 240 241 242 243 244 245 246 247 248 249
 250 251 252 253 254 255 288 289 290 291 292 293 294 295 296 297 298 299
 300 301 302 303 336 337 338 339 340 341 342 343 344 345 346 347 348 349
 350 351 384 385 386 387 388 389 390 391 392 393 394 395 396 397 398 399
 432 433 434 435 436 437 438 439 440 441 442 443 444 445 446 447]


In [104]:
# バイアスの勾配．バイアスはチャンネルごとの計算 = 列方向の和をとる
db = np.sum(dout, axis=0)
print('dbの形状:', db.shape, end='\n\n')


# フィルターの勾配．x^Tとの積を取ってフィルターの形状に戻す
dW = np.dot(col.T, dout)
print('np.dot(col.T, dout)の形状:', dW.shape)
dW = dW.transpose(1, 0).reshape(3, 3, 2, 2)
print('dWの形状:', dW.shape, end='\n\n')


# 入力の勾配．Wとの積を取って，元の形状に戻す
dcol = np.dot(dout, col_w.T)
print('np.dot(dout, col_w.T)の形状:', dcol.shape)
dx = col2im(dcol, x.shape, 2, 2, stride=1, pad=0)
print('dxの形状:', dx.shape, end='\n\n')

dbの形状: (3,)

np.dot(col.T, dout)の形状: (12, 3)
dWの形状: (3, 3, 2, 2)

np.dot(dout, col_w.T)の形状: (160, 12)
dxの形状: (10, 3, 5, 5)



### 実装
- W: フィルター．4次元(FN, C, FH, FW)
- b: バイアス
- x: 入力．4次元(N, C, H, W)
- backward は 以下Affineレイヤを参考

$$
\frac{\partial L}{\partial W} = \boldsymbol{x}^T \frac{\partial L}{\partial \boldsymbol{y}}
\\\\
\frac{\partial L}{\partial \boldsymbol{x}} = \frac{\partial L}{\partial \boldsymbol{y}} W^T
$$

- $\frac{\partial L}{\partial \boldsymbol{y}}$ は，逆伝播の入力として与えられる．形状は (N, FN, OH, OW) -> (N, OH, OW, FN) -> ((N x OH x OW), FN) と変換
- $\boldsymbol{x}$ は，im2col適用後の入力データに相当．形状は ((N x OH x OW), (C x FH x FW))
- $W$ は，展開後のフィルターに相当．形状は ((C x FH x FW), FN)

In [24]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        # フィルターと入力データの形状
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        
        # フィルターと入力データの形状から，出力データの形状を計算
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        # フィルターサイズ，ストライド，パディングを考慮して入力データを2次元に展開
        col = im2col(x, FH, FW, self.stride, self.pad)
        # 1個のフィルターを1列に展開
        col_W = self.W.reshape(FN, -1).T
        
        # 畳み込み演算 ((N x out_h x out_w), (C x FH x FW)) x ((C x FH x FW)), FN) => ((N x out_h x out_w), FN)
        out = np.dot(col, col_W) + self.b
        
        # ((N x out_h x out_w), FN) => (N, FN, out_h, out_w)
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out
    
    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        # forwardの出力と同じサイズ(N, FN, out_h, out_w) => ((N x out_h x out_w), FN)
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx

# Poolingレイヤの実装
- im2col でデータを展開するが，チャンネル方向には独立な処理
- 行ごとに最大値を求めて形状を整形する

In [ ]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        # プーリング適用領域を1行に展開
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad) # この段階ではチャンネル方向も1行にまとまっている
        col = col.reshape(-1, self.pool_h*self.pool_w)                   # 列数をプーリング領域の高さ x 幅で区切る
        
        # 各行で最大値取得
        out = np.max(col, axis=1)
        
        # 整形(3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out

#### プーリングの整形流れ

In [118]:
a = np.arange(1*3*4*4).reshape(1, 3, 4, 4)
a

array([[[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [12, 13, 14, 15]],

        [[16, 17, 18, 19],
         [20, 21, 22, 23],
         [24, 25, 26, 27],
         [28, 29, 30, 31]],

        [[32, 33, 34, 35],
         [36, 37, 38, 39],
         [40, 41, 42, 43],
         [44, 45, 46, 47]]]])

In [122]:
b = im2col(a, 2, 2, stride=2, pad=0)
b

array([[ 0.,  1.,  4.,  5., 16., 17., 20., 21., 32., 33., 36., 37.],
       [ 2.,  3.,  6.,  7., 18., 19., 22., 23., 34., 35., 38., 39.],
       [ 8.,  9., 12., 13., 24., 25., 28., 29., 40., 41., 44., 45.],
       [10., 11., 14., 15., 26., 27., 30., 31., 42., 43., 46., 47.]])

In [123]:
c = b.reshape(-1, 2*2)
c

array([[ 0.,  1.,  4.,  5.],
       [16., 17., 20., 21.],
       [32., 33., 36., 37.],
       [ 2.,  3.,  6.,  7.],
       [18., 19., 22., 23.],
       [34., 35., 38., 39.],
       [ 8.,  9., 12., 13.],
       [24., 25., 28., 29.],
       [40., 41., 44., 45.],
       [10., 11., 14., 15.],
       [26., 27., 30., 31.],
       [42., 43., 46., 47.]])

In [124]:
d = np.max(c, axis=1)
d

array([ 5., 21., 37.,  7., 23., 39., 13., 29., 45., 15., 31., 47.])

In [125]:
d.reshape(1, 2, 2, 3)

array([[[[ 5., 21., 37.],
         [ 7., 23., 39.]],

        [[13., 29., 45.],
         [15., 31., 47.]]]])

In [126]:
d.reshape(1, 2, 2, 3).transpose(0, 3, 1, 2)

array([[[[ 5.,  7.],
         [13., 15.]],

        [[21., 23.],
         [29., 31.]],

        [[37., 39.],
         [45., 47.]]]])